# Setup

In [1]:
import contracts
contracts.disable_all()

In [2]:
import duckietown_world as dw
from duckietown_world.svg_drawing.ipython_utils import ipython_draw_html

INFO:dt-world:duckietown-world 1.0.15
DEBUG:dt-serialization:Registering class Serializable
DEBUG:dt-serialization:Registering class Serializable
DEBUG:dt-serialization:Registering class GenericData
DEBUG:dt-serialization:Registering class Sequence
DEBUG:dt-serialization:Registering class SampledSequence
DEBUG:dt-serialization:Registering class Constant
DEBUG:dt-serialization:Registering class RectangularArea
DEBUG:dt-serialization:Registering class TransformSequence
DEBUG:dt-serialization:Registering class VariableTransformSequence
DEBUG:dt-serialization:Registering class SE2Transform
DEBUG:dt-serialization:Registering class Scale2D
DEBUG:dt-serialization:Registering class Matrix2D
DEBUG:dt-serialization:Registering class SpatialRelation
DEBUG:dt-serialization:Registering class GroundTruth
DEBUG:dt-serialization:Registering class PlacedObject
DEBUG:dt-serialization:Registering class EvaluatedMetric
DEBUG:dt-serialization:Registering class GenericObject
DEBUG:dt-serialization:Registeri

In [3]:
dw.logger.setLevel(50)

In [4]:
%%html
<style>
pre {line-height: 90%}
</style>

# Map

In [5]:
m = dw.load_map('robotarium1')

In [6]:
sk = dw.get_skeleton_graph(m)

# Example for a path

In [7]:
start = 'P60'
end = 'P36'

In [8]:
import networkx as nx
path = nx.shortest_path(sk.G, start, end)
print(path)

['P60', 'P23', 'P41', 'P62', 'P32', 'P20', 'P67', 'P39', 'P36']


# Cost function 

In [9]:
def cost_function():
    import random as rn
    return rn.randint(0,10)
    

# Function that creates velocity profiler graph

In [10]:
def get_lanes(path, G):
    edges = zip(path[:-1], path[1:]) 
    lanes  = []
    for a, b in edges:
        lane = G.get_edge_data(a, b)[0]['lane']
        lanes.append(lane)
    return lanes

In [11]:
def velocity_profiler(path, G):
    import networkx as nx
    import numpy as np
    
    vel_G = nx.DiGraph()
    vel_min = 0.1
    vel_max = 0.5
    vel_space = np.linspace(vel_min, vel_max, 5)
    
#     lanes = get_lanes(path);
    
    vel_G.add_node((1,1));
    
    for i in range(len(path)-1):
        for j, vel in enumerate(vel_space):
            vel_G.add_node((i+2, j+1))
        
    for (i,j) in vel_G:
        if i==1:
            for k, vel in enumerate(vel_space):
                vel_G.add_edge((1,1),(2,k+1), weight = cost_function())
        elif i<len(path):
            for k, vel in enumerate(vel_space):
                vel_G.add_edge((i,j),(i+1,k+1), weight = cost_function())
                
            
            
                
    
#     for j, vel in enumerate(vel_space):
#         vel_G.add_edge((1, 1), (2,j), weight = cost_function())
        
#     for i in range(len(lanes)-2):
#         for j, vel in enumerate(vel_space):
#             vel_G.add_edge('node'+str(i+2)+'vel'+str(j+1),'node'+str(i+3)+'vel'+str(j+1),
#                            weight = cost_function())
#     for i, lane in enumerate(lanes):
#         for j, vel in enumerate(vel_space):
#             vel_G.addedge('node'+str(i)+,j, weight = cost_function(), velocity = vel)
            
    return vel_G
        

In [12]:
def draw_graph(G0, pos=None):
    from matplotlib import pyplot as plt
    pos = pos or nx.spring_layout(G0)# so nodes are nicely separated
    plt.figure(figsize=(12, 12))    
    nx.draw(G0,pos,labels={node:node for node in G0.nodes()})
#     def edge_label(a, b):
#         datas = G0.get_edge_data(a, b)
#         s = '%d edge%s' % (len(datas), 's' if len(datas)>=2 else '')
#         for k, v in datas.items():
#             if v:
#                 if 'label' in v:
#                     s += '\n %s' % v['label']
#                 else:
#                     s += '\n %s' %v
#         return s
#     edge_labels = dict([ ((a,b), edge_label(a,b)) for a,b in G0.edges()])
#     nx.draw_networkx_edge_labels(G0,pos,edge_labels=edge_labels,font_color='red')
    plt.axis('off')
    plt.show()

In [13]:
vel_G = velocity_profiler(path, sk.G)

In [14]:
draw_graph(vel_G)

<Figure size 1200x1200 with 1 Axes>

In [15]:
vel_path = nx.astar_path(vel_G, (1, 1), (len(path), 2), heuristic=None, weight='weight')

In [16]:
import numpy as np

In [17]:
vel_min = 0.1
vel_max = 0.5
vel_space = np.linspace(vel_min, vel_max, 5)

In [18]:
for k, node in enumerate(vel_path):
    print('k: {}, node: {}, velocity: {}'.format(k, node, vel_space[node[1]-1]))

k: 0, node: (1, 1), velocity: 0.1
k: 1, node: (2, 5), velocity: 0.5
k: 2, node: (3, 1), velocity: 0.1
k: 3, node: (4, 2), velocity: 0.2
k: 4, node: (5, 5), velocity: 0.5
k: 5, node: (6, 4), velocity: 0.4
k: 6, node: (7, 1), velocity: 0.1
k: 7, node: (8, 1), velocity: 0.1
k: 8, node: (9, 2), velocity: 0.2
